# LDA using Gibb's sampling

Op basis van uitleg in http://u.cs.biu.ac.il/~89-680/darling-lda.pdf

Using Gensim functionality

Dus we mogen ervan uitgaan dat we een dictionary hebben en bow vectors voor de documenten.
Het een en ander aan counts komt dan uit gensim. Maar hoe?

Wat hebben we nodig?

* The number of words assigned to topic k in document d: $$n_{d;k}$$
* The number of times word w is assigned to topic k: $$n_{k;w}$$
* The total number of times any word is assigned to topic k: $$n_k$$
* Array z which will contain the current topic assignment for each of the N words in the corpus.


In [1]:
# Generate documents
import numpy as np

length = 100
num_topics = 3
vocabulary = np.array(['zon', 'ijs', 'strand', 'vanille', 'chocola', 'broccoli', 'wortel'])

real_theta = np.array([[0.5, 0.0, 0.5],
                       [0.3, 0.3, 0.4],
                       [0.1, 0.8, 0.1],
                       [0.7, 0.3, 0.0],
                       [0.4, 0.4, 0.2]])
real_phi = np.array([[0.4, 0.2, 0.4, 0.0, 0.0, 0.0, 0.0],
                     [0.0, 0.3, 0.0, 0.35, 0.35, 0.0, 0.0],
                     [0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.5]])

In [6]:
# generate the corpus
corpus = []

for m, tm in enumerate(real_theta):
    doc = []
    for i in range(length):
        # sample topic
        topic = np.random.multinomial(1, tm)
        topic_index = np.where(topic==1)[0][0]
        word = np.random.multinomial(1, real_phi[topic_index])
        doc.append(vocabulary[np.where(word==1)[0][0]])
    corpus.append(doc)
    
for i, doc in enumerate(corpus):
    print 'document', i+1, doc



document 1 ['wortel', 'strand', 'wortel', 'broccoli', 'wortel', 'broccoli', 'wortel', 'broccoli', 'zon', 'broccoli', 'broccoli', 'zon', 'wortel', 'strand', 'broccoli', 'broccoli', 'wortel', 'strand', 'strand', 'strand', 'wortel', 'strand', 'ijs', 'wortel', 'wortel', 'wortel', 'strand', 'broccoli', 'broccoli', 'wortel', 'wortel', 'zon', 'broccoli', 'wortel', 'strand', 'strand', 'ijs', 'wortel', 'wortel', 'strand', 'strand', 'ijs', 'broccoli', 'broccoli', 'zon', 'wortel', 'broccoli', 'zon', 'ijs', 'ijs', 'wortel', 'broccoli', 'zon', 'zon', 'zon', 'zon', 'strand', 'zon', 'broccoli', 'broccoli', 'broccoli', 'broccoli', 'wortel', 'wortel', 'zon', 'broccoli', 'zon', 'ijs', 'zon', 'zon', 'strand', 'ijs', 'wortel', 'strand', 'wortel', 'zon', 'ijs', 'wortel', 'wortel', 'strand', 'strand', 'broccoli', 'zon', 'zon', 'zon', 'strand', 'broccoli', 'broccoli', 'strand', 'strand', 'broccoli', 'ijs', 'broccoli', 'wortel', 'wortel', 'zon', 'strand', 'wortel', 'broccoli', 'wortel']
document 2 ['wortel', 

In [31]:
from gensim import corpora

d = corpora.Dictionary(corpus)

In [7]:
# initialize z
import random
from collections import Counter

z = {}
ndk = np.zeros((len(corpus), len(real_phi)), dtype=np.int)
nkw = np.zeros((len(real_phi), len(vocabulary)), dtype=np.int)
nk = Counter()

for d, doc in enumerate(corpus):
    for i, word in enumerate(doc):
        t = random.randint(0,2)
        z[(d, i)] = t
        ndk[d][t] += 1
        word_index = np.where(vocabulary==word)[0][0]
        nkw[t][word_index] += 1
        nk[t] += 1

print z
print ndk
print np.sum(ndk)
print nkw
print np.sum(nkw)
print nk

{(0, 86): 0, (3, 35): 1, (4, 36): 0, (2, 84): 1, (0, 76): 0, (4, 66): 0, (1, 64): 1, (2, 78): 0, (0, 98): 1, (3, 86): 1, (0, 17): 1, (1, 28): 1, (2, 27): 2, (0, 55): 1, (3, 2): 1, (1, 54): 2, (4, 5): 1, (2, 53): 1, (0, 45): 2, (3, 40): 1, (1, 40): 1, (4, 35): 2, (2, 47): 0, (0, 67): 0, (4, 73): 0, (1, 89): 1, (2, 73): 0, (3, 95): 0, (4, 87): 2, (2, 99): 1, (1, 21): 2, (2, 18): 0, (0, 14): 0, (3, 11): 2, (1, 15): 2, (4, 12): 1, (2, 12): 1, (0, 36): 2, (3, 17): 2, (1, 33): 1, (4, 42): 0, (2, 38): 2, (0, 90): 0, (3, 55): 2, (4, 48): 2, (1, 82): 0, (2, 64): 0, (3, 68): 1, (4, 94): 0, (0, 5): 2, (1, 0): 2, (4, 11): 0, (2, 7): 0, (0, 59): 2, (3, 22): 1, (1, 58): 1, (4, 17): 0, (2, 33): 0, (0, 81): 0, (3, 60): 2, (4, 63): 2, (2, 91): 2, (3, 77): 0, (4, 69): 2, (1, 77): 1, (3, 83): 0, (4, 99): 2, (0, 28): 0, (1, 25): 1, (2, 30): 2, (0, 50): 1, (3, 31): 1, (1, 51): 1, (4, 24): 0, (2, 56): 1, (0, 40): 2, (3, 37): 2, (4, 38): 0, (2, 82): 0, (0, 78): 0, (4, 76): 1, (1, 70): 0, (2, 76): 1, (3, 88):

In [8]:
# iterate

def p_z(alpha, beta, num_topics, num_words, d, topic, word_index):
    #print alpha, beta, num_topics, num_words, d, topic, word_index
    #print ndk[d][topic], nkw[topic][word_index], nk[topic]
    return (ndk[d][topic]*alpha)*(nkw[topic][word_index]+beta)/(nk[topic]+beta*num_words)

def normalize(p):
    minimum = np.min(p)
    maximum = np.max(p)
    
    if minimum < 0:
        return normalize((p - minimum)/(maximum - minimum))
    return p/sum(p)

num_iter = 10000

alpha = 5
beta = 0.02

theta = np.zeros((num_iter,  len(corpus), num_topics))
phi = np.zeros((num_iter, num_topics, len(vocabulary)))

for t in range(num_iter):
    for d, doc in enumerate(corpus):
        for i, w in enumerate(doc):
            word = (d, i)
            topic = z[word]
            
            #print w
            
            #print 'prev topic', topic
            # mag alleen als de betreffende getallen > 0, anders komen er negatieve kansen uit (en dat kan niet)
            # vergelijk met https://gist.github.com/mblondel/542786
            
            #if ndk[d][topic] == 0 or nkw[topic][word_index] == 0 or nk[topic] == 0:
            #    print 'Found a zero'
            #    print 'ndk', ndk[d][topic]
            #    print ndk
            #    print
            #    print 'nkw', nkw[topic][word_index]
            #    print nkw
            #    print
                
            #    print 'nk', nk[topic]
            #    print nk
            
            word_index = np.where(vocabulary==w)[0][0]
                
            if ndk[d][topic] > 0 and nkw[topic][word_index] > 0 and nk[topic] > 0:
                ndk[d][topic] -= 1
                nkw[topic][word_index] -= 1
                nk[topic] -= 1
            
                p = [p_z(alpha, beta, len(real_phi), len(vocabulary), d, i, word_index) for i in range(len(real_phi))]
                #print p
                # normalize
                p = normalize(p)
                #print p, sum(p)
            
                to = np.random.multinomial(1, p)
                #except ValueError:
                #    print 
                #    print p
                #    print alpha, beta, len(real_phi), len(vocabulary), d, i, word_index
                #    print ndk[d][topic], nkw[topic][word_index], nk[topic]
                topic = np.where(to==1)[0][0]
            
                ndk[d][topic] += 1
                nkw[topic][word_index] += 1
                nk[topic] += 1
    # calculate theta and phi
    theta[t] = (ndk+float(alpha))/(np.sum(ndk, axis=1, keepdims=True)+num_topics*alpha)
    phi[t] = (nkw+float(beta))/(np.sum(nkw, axis=1, keepdims=True)+len(vocabulary)*beta)
            #print 'new topic', topic
print ndk
print nkw
print nk

[[63  0 37]
 [54 46  0]
 [ 0 35 65]
 [ 0 76 24]
 [25  0 75]]
[[80  0  0  0  0 62  0]
 [ 0 98  0  0 59  0  0]
 [ 0  0 82 57  0  0 62]]
Counter({2: 201, 1: 157, 0: 142})


In [9]:
print 'theta'
print 'found'
print np.mean(theta, axis=0)
print 'real'
print real_theta
print
print 'phi'
print 'found'
print np.mean(phi, axis=0)
print 'real'
print real_phi
print
print 'topics found:'
indexes = np.mean(phi, axis=0) > 0.01
for index in indexes:
    print vocabulary[index]
print
print 'topics real'
indexes = real_phi > 0.01
for index in indexes:
    print vocabulary[index]

theta
found
[[ 0.57306087  0.04359304  0.38334609]
 [ 0.47332435  0.48317217  0.04350348]
 [ 0.04349652  0.39480957  0.56169391]
 [ 0.08858261  0.61784435  0.29357304]
 [ 0.27371043  0.04354348  0.68274609]]
real
[[ 0.5  0.   0.5]
 [ 0.3  0.3  0.4]
 [ 0.1  0.8  0.1]
 [ 0.7  0.3  0. ]
 [ 0.4  0.4  0.2]]

phi
found
[[  5.62894690e-01   1.69806329e-04   1.55745091e-04   1.54608352e-04
    1.46561809e-04   4.36314282e-01   1.64306498e-04]
 [  1.39052791e-04   6.23605296e-01   2.17692036e-04   1.74437224e-04
    3.75506031e-01   1.35017200e-04   2.22474328e-04]
 [  1.59071782e-04   1.37276712e-04   4.07734811e-01   2.83484857e-01
    1.07186624e-04   1.06691917e-04   3.08270105e-01]]
real
[[ 0.4   0.2   0.4   0.    0.    0.    0.  ]
 [ 0.    0.3   0.    0.35  0.35  0.    0.  ]
 [ 0.    0.    0.    0.    0.    0.5   0.5 ]]

topics found:
['zon' 'broccoli']
['ijs' 'chocola']
['strand' 'vanille' 'wortel']

topics real
['zon' 'ijs' 'strand']
['ijs' 'vanille' 'chocola']
['broccoli' 'wortel']
